In [1]:
using QuadGK
using Plots
plotly();

ArgumentError: ArgumentError: Package QuadGK not found in current path:
- Run `import Pkg; Pkg.add("QuadGK")` to install the QuadGK package.


In [2]:
function f(x)
    return 1.0 / (25x^2+1)
end

f (generic function with 1 method)

In [3]:
function printPoly(coefs, newline=true, symbol="x")
    coefs = reverse(coefs)
    separators = [""; (x -> (isa(x, Number) && x < 0 ? "" : "+")).(coefs[2:end])]
    symbols = [(x -> string(symbol, "^", x)).(length(coefs)-1:-1:1); ""]
    print(join(Iterators.flatten(zip(separators, coefs, symbols))), newline ? '\n' : "")
end

printPoly (generic function with 3 methods)

In [4]:
printPoly([-1.0,2.0,-3.0,4.0,-5.0])
printPoly([-1.0, 'c', 'b', 2.0, 'a'])

-5.0x^4+4.0x^3-3.0x^2+2.0x^1-1.0
ax^4+2.0x^3+bx^2+cx^1-1.0


In [5]:
function createPolynomial(coef)
    return x -> horner(x, coef)
end

createPolynomial (generic function with 1 method)

In [6]:
#funkcja usuwająca z tablicy wartości bliskie zera
function chopZeros(array, eps=1e-10)
    array[abs.(array) .<= eps] .= 0
end

chopZeros (generic function with 2 methods)

In [7]:
legendreBasis = 
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.;
0. 1. 0. 0. 0. 0. 0. 0. 0. 0.;
-0.5 0. 1.5 0. 0. 0. 0. 0. 0. 0.;
0. -1.5 0. 2.5 0. 0. 0. 0. 0. 0.;
0.375 0. -3.75 0. 4.375 0. 0. 0. 0. 0.;
0. 1.875 0. -8.75 0. 7.875 0. 0. 0. 0.;
-0.3125 0. 6.5625 0. -19.6875 0. 14.4375 0. 0. 0.;
0. -2.1875 0. 19.6875 0. -43.3125 0. 26.8125 0. 0.;
0.273438 0. -9.84375 0. 54.1406 0. -93.8438 0. 50.2734 0.;
0. 2.46094 0. -36.0938 0. 140.766 0. -201.094 0. 94.9609];

In [8]:
chebyshevBasis = 
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.;
0. 1. 0. 0. 0. 0. 0. 0. 0. 0.;
-1. 0. 2. 0. 0. 0. 0. 0. 0. 0.;
0. -3. 0. 4. 0. 0. 0. 0. 0. 0.;
1. 0. -8. 0. 8. 0. 0. 0. 0. 0.;
0. 5. 0. -20. 0. 16. 0. 0. 0. 0.;
-1. 0. 18. 0. -48. 0. 32. 0. 0. 0.;
0. -7. 0. 56. 0. -112. 0. 64. 0. 0.;
1. 0. -32. 0. 160. 0. -256. 0. 128. 0.;
0. 9. 0. -120. 0. 432. 0. -576. 0. 256.];

In [9]:
legendreWeight = x -> 1;

In [10]:
legendreOrthoCoef = k -> 1/(2k+1);

In [11]:
chebyshevWeight = x-> 1/sqrt(1-x^2);

In [12]:
chebyshevOrthoCoef = k -> (k == 0 ? pi : pi/2);

In [13]:
function horner(x::T, coefs::Array{Float64, 1})::T where {T}
    result::T = coefs[end]
    for i = length(coefs)-1:-1:1
        result = result*x + coefs[i]
    end
    return result
end

horner (generic function with 1 method)

In [14]:
function newtonCoefs(f, nodes)
    coefs = zeros(length(nodes))
    for k = 1:length(nodes)
        for i = 1:k
            tmp = f(nodes[i])
            for j = 1:k
                if j == i continue end
                tmp /= nodes[i] - nodes[j]
            end
            coefs[k] += tmp
        end
    end
    return coefs
end

function newton(x, nodes, coefs)
    p = [1; cumprod(x .- nodes[1:end-1])]
    return sum(p .* coefs)
end

function createNewtonInterpolant(f, nodes)
    return x -> newton(x, nodes, newtonCoefs(f, nodes))
end

createNewtonInterpolant (generic function with 1 method)

In [15]:
function chebyshevZeros(n::Int)
    return chebyshevZeros(collect(0:n-1))
end

function chebyshevZeros(a::Array{Int,1})
    return chebyshevZeros.(length(a)-1, a)
end

function chebyshevZeros(n, k)
    return cos((2k+1)/(2n+2)*pi)
end

chebyshevZeros (generic function with 3 methods)

In [16]:
function chebyshevExtrema(n::Int)
    return chebyshevExtrema(collect(0:n))
end

function chebyshevExtrema(a::Array{Int,1})
    return chebyshevExtrema.(length(a)-1, a)
end

function chebyshevExtrema(n, k)
    return cos(k/n*pi)
end

chebyshevExtrema (generic function with 3 methods)

In [17]:
function findMaxDev(f, w, N=1000)
    nodes = -1 .+ 2 .* collect(0:N) ./ N
    deviations = abs.((x -> (f(x) - w(x))).(nodes))
    return maximum(deviations)
end

findMaxDev (generic function with 2 methods)

In [18]:
function leastSquares(f, n, basisMatrix, orthoCoef, w=x->1)
    c = orthoCoef.(0:n)
    a = (k -> quadgk(x -> (w(x)*f(x)*createPolynomial(basisMatrix[k, 1:k])(x)), -1, 1, atol=0.00001)[1]).(1:n+1) ./ c
    coefs = sum(a .* basisMatrix[1:n+1,1:n+1], dims=1)[:]
    
    chopZeros(coefs)
    return coefs
end

leastSquares (generic function with 2 methods)

In [19]:
function main()
    optimalLegendre = leastSquares(f, 9, legendreBasis, legendreOrthoCoef, legendreWeight)
    optimalLegendreApproximant = createPolynomial(optimalLegendre)

    optimalChebyshev = leastSquares(f, 9, chebyshevBasis, chebyshevOrthoCoef, chebyshevWeight)
    optimalChebyshevApproximant = createPolynomial(optimalChebyshev)

    equidistantNodes = range(-1, stop = 1, length = 20)
    equidistantInterpolant = createNewtonInterpolant(f, equidistantNodes)

    chebyshevZerosNodes = chebyshevZeros(20)
    chebyshevZerosInterpolant = createNewtonInterpolant(f, chebyshevZerosNodes)

    chebyshevExtremaNodes = chebyshevExtrema(19)
    chebyshevExtremaInterpolant = createNewtonInterpolant(f, chebyshevExtremaNodes)
    
    
    println("Legendre ", findMaxDev(f, optimalLegendreApproximant, 10000))
    println("Chebyshev ", findMaxDev(f, optimalChebyshevApproximant, 10000))
    println("Equidistant ", findMaxDev(f, equidistantInterpolant, 10000))
    println("Chebyshev Zeros ", findMaxDev(f, chebyshevZerosInterpolant, 10000))
    println("Chebyshev Extrema ", findMaxDev(f, chebyshevExtremaInterpolant, 10000))
    
    rng = range(-1.1, stop = 1.1, length = 200)
    plot(rng, f.(rng), label="f", size=(600*1.5,400*1.5), ylim=(-1, 2.5), xlim=(-1.1, 1.1))
    plot!(rng, optimalLegendreApproximant.(rng), linestyle=:dot, label="Legendre")
    plot!(rng, optimalChebyshevApproximant.(rng), linestyle=:dot, label="Chebyshev")
    plot!(rng, equidistantInterpolant.(rng), linestyle=:dash, label="Equidistant")
    plot!(rng, chebyshevZerosInterpolant.(rng), linestyle=:dashdot, label="Chebyshev Zeros")
    plot!(rng, chebyshevExtremaInterpolant.(rng), linestyle=:dashdot, label="Chebyshev Extrema")
end

main()

UndefVarError: UndefVarError: quadgk not defined